In [ ]:
import mdirac as moire
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
import shutil
import json
import matplotlib.pyplot as plt
import time

###
### MoS2 A exciton calculation on SiO2
###

a = 3.16 # 
a0 = 0.52917 # [A] hydrogen
shiftA = 0.71
Eg = 1.585+shiftA # [eV] wu (2015) + rigid shift to achieve A exciton at 1.85 eV
Ry = 13.60565 # [eV] hydrogen
eps = 2.5 # silicon substrate
r0 = 33.875/2.5 # [A] wu (2015)
m = 0.58

Ry = Ry/Eg
t = np.sqrt(0.5*Ry*(1/m))*a0/a # in units of Eg
teff = 2*np.pi*t # 2pi from 2pi/a in monolayer hamiltonian

#keldysh interaction parameters
r0eff = 2*np.pi*r0/a
epseff = eps/Ry*a/a0/2

#optical slab model for transition from sigma to epsilion
d = 6.5 # [A], MoS2 thickness
beta = Ry*2*a0/d 

#grid density 
kstep0 = 0.003
scale=0.5
#cell=kstep0*np.array([[1.,0.],[-0.5,0.5*np.sqrt(3)]])
# complicable with moire hamiltonian reciprocal vectors
cell=kstep0*np.array([[0.5*np.sqrt(3),0.5],[-0.5*np.sqrt(3),0.5]])
N = int(1/kstep0)
kmesh = moire.mesh.Parallelogram(N,cell)

deg = 1 # valley degeneracy is gone because of lcp or rcp
# spin degeneracy is gone because of spin-splitted valence band

def mkdir_p(mypath):
    from errno import EEXIST
    try:
        os.makedirs(mypath)
    except OSError as exc:
        if exc.errno == EEXIST and os.path.isdir(mypath):
            pass
        else: raise

path = 'out/MoS2_AB/'+'scale'+str(scale)+'kstep0_'+str(kstep0)
mkdir_p(path)

circle = kmesh.get_BZ(scale=scale)
meshpath = '/mesh.png'
circle.savefig(path=(path+meshpath))

f = open('mono.ipynb')
data = json.load(f)
log = data['cells'][0]['source']
f.close()
out_f = open(path+"/log.json", "w")
json.dump(log, out_f, indent = 6)
out_f.close()

timelog = open(path+"/timelog_A.txt", "w")
startTime = time.time()
E,U = moire.mdirac.eighMesh(circle,teff)
executionTime = (time.time() - startTime)
timelog.write("Hamiltonian diagonalization time: " + str(executionTime) + '\n')


plt.plot(np.arange(E[:,1].shape[0]),E[:,1])
plt.savefig(path+'/disps_c_A.png',dpi=100)
plt.close()

plt.plot(np.arange(E[:,0].shape[0]),E[:,0])
plt.savefig(path+'/disps_v_A.png',dpi=100)
plt.close()

keld=moire.interaction.Keldysh(epseff,r0eff,kstep0)
Wk=moire.interaction.sample_to_mesh(keld,kmesh)
Wklog = open(path+"/Wklog_A.txt", "w")
Wklog.write('k space, singularity resolved with integration over parallelogram -> 4.254' + '\n')
Wklog.write('Wk[0,0] '+str(Wk[0,0]) + '\n')
Wklog.write('Wk[0,1] '+str(Wk[0,1]))
Wklog.close()

keldpath = '/keld_A.png'
kmesh.savefig(c=np.log(np.abs(Wk)),path=(path+keldpath),dpi=100,s=100)

startTime = time.time()
Wkk=moire.interaction.pairs(circle,Wk)
executionTime = (time.time() - startTime)
timelog.write("Keldysh pairing time: " + str(executionTime) + '\n')

Ex=np.zeros((circle.Np,2,2),dtype=np.float64)
Ux=np.zeros((circle.Np,circle.Np,2,2),dtype=np.complex128)

startTime = time.time()
H = moire.mdirac.Hbse(E,U,Wkk)
Ex, Ux = np.linalg.eigh(H)
executionTime = (time.time() - startTime)
timelog.write("BSE diagonalization time: " + str(executionTime) +'\n')

Exlog = open(path+"/Exlog_A.txt", "w")
Exlog.write('Exciton energies in energy units of gap: \n')
Exlog.write(str(Ex[:5]) + '\n')
Exlog.write('Exciton energies in eV energy units: \n')
Exlog.write(str(Eg*Ex[:5]) + '\n')
Eb=Eg*(1.-Ex[:5])
Exlog.write('Exciton binding energies in eV: \n')
Exlog.write(str(Eb[:5]))
Exlog.close()

startTime = time.time()
sx=np.array([[0,1],[1,0]])
sy=np.array([[0,1.j],[-1.j,0]])

factorsigma=deg*kmesh.vcell*2*Ry*(a0/a)**2*(1/m)*1.j
omega=np.linspace(1.e-5,2.,1000)
eta=1.5e-2
csxv=np.zeros((circle.Np),dtype=np.complex)
xsx0=np.zeros((circle.Np),dtype=np.complex)
csyv=np.zeros((circle.Np),dtype=np.complex)
xsy0=np.zeros((circle.Np),dtype=np.complex)
dE=np.zeros((circle.Np),dtype=np.float64)

sigma=np.zeros((omega.shape[0]),dtype=np.complex128)
sigma0=np.zeros((omega.shape[0]),dtype=np.complex128)

csxv = moire.mdirac.cov_matrix_elements(sx,U,1,0)
xsx0 = moire.mdirac.exciton_elements(Ux,csxv)
csyv = moire.mdirac.cov_matrix_elements(sy,U,1,0)
xsy0 = moire.mdirac.exciton_elements(Ux,csyv)
dE = E[:,1]-E[:,0]

sigma0x=moire.mdirac.get_sigma_xx(factorsigma,omega+1.j*eta,dE,csxv)
sigmaxx=moire.mdirac.get_sigma_xx(factorsigma,omega+1.j*eta,Ex,xsx0)
sigma0y=moire.mdirac.get_sigma_xy(factorsigma,omega+1.j*eta,dE,csxv,csyv)
sigmaxy=moire.mdirac.get_sigma_xy(factorsigma,omega+1.j*eta,Ex,xsx0,xsy0)
sigma_rcp0_A = sigma0x+1.j*sigma0y
sigma_rcp_A = sigmaxx+1.j*sigmaxy
sigma_lcp0 = sigma0x-1.j*sigma0y
sigma_lcp = sigmaxx-1.j*sigmaxy

eps_A = 4*np.pi*sigma_rcp_A/omega*beta*1.j

executionTime = (time.time() - startTime)
timelog.write("Matrix element calculation: " + str(executionTime))

fsumlog = open(path+"/fsum.txt", "w")
fsumlog.write('Transition sum rules for different transitions in order 0123 -> 12,13,02,03: \n')
fsumlog.write(str(np.sum(np.abs(csxv)**2))+'\t'+str(np.sum(np.abs(xsx0)**2))+'\n')
fsumlog.close()

###
### MoS2 B exciton calculation on SiO2
###

a = 3.16 # 
a0 = 0.52917 # [A] hydrogen
shiftB = 0.16
Eg = 1.585+shiftA+shiftB # [eV] wu (2015) + rigid shift to achieve A exciton at 1.85 eV 
#+ rigid shift to B exciton
Ry = 13.60565 # [eV] hydrogen
eps = 2.5 # silicon substrate
r0 = 33.875/2.5 # [A] wu (2015)
m = 0.54

Ry = Ry/Eg
t = np.sqrt(0.5*Ry*(1/m))*a0/a # in units of Eg
teff = 2*np.pi*t # 2pi from 2pi/a in monolayer hamiltonian

#keldysh interaction parameters
r0eff = 2*np.pi*r0/a
epseff = eps/Ry*a/a0/2

#optical slab model for transition from sigma to epsilion
d = 6.5 # [A], MoS2 thickness
beta = Ry*2*a0/d 

timelog = open(path+"/timelog_B.txt", "w")
startTime = time.time()
E,U = moire.mdirac.eighMesh(circle,teff)
executionTime = (time.time() - startTime)
timelog.write("Hamiltonian diagonalization time: " + str(executionTime) + '\n')

plt.plot(np.arange(E[:,1].shape[0]),E[:,1])
plt.savefig(path+'/disps_c_B.png',dpi=100)
plt.close()

plt.plot(np.arange(E[:,0].shape[0]),E[:,0])
plt.savefig(path+'/disps_v_B.png',dpi=100)
plt.close()

keld=moire.interaction.Keldysh(epseff,r0eff,kstep0)
Wk=moire.interaction.sample_to_mesh(keld,kmesh)
Wklog = open(path+"/Wklog_B.txt", "w")
Wklog.write('k space, singularity resolved with integration over parallelogram -> 4.254' + '\n')
Wklog.write('Wk[0,0] '+str(Wk[0,0]) + '\n')
Wklog.write('Wk[0,1] '+str(Wk[0,1]))
Wklog.close()

keldpath = '/keld_B.png'
kmesh.savefig(c=np.log(np.abs(Wk)),path=(path+keldpath),dpi=100,s=100)

startTime = time.time()
Wkk=moire.interaction.pairs(circle,Wk)
executionTime = (time.time() - startTime)
timelog.write("Keldysh pairing time: " + str(executionTime) + '\n')

Ex=np.zeros((circle.Np,2,2),dtype=np.float64)
Ux=np.zeros((circle.Np,circle.Np,2,2),dtype=np.complex128)

startTime = time.time()
H = moire.mdirac.Hbse(E,U,Wkk)
Ex, Ux = np.linalg.eigh(H)
executionTime = (time.time() - startTime)
timelog.write("BSE diagonalization time: " + str(executionTime) +'\n')

Exlog = open(path+"/Exlog_B.txt", "w")
Exlog.write('Exciton energies in energy units of gap: \n')
Exlog.write(str(Ex[:5]) + '\n')
Exlog.write('Exciton energies in eV energy units: \n')
Exlog.write(str(Eg*Ex[:5]) + '\n')
Eb=Eg*(1.-Ex[:5])
Exlog.write('Exciton binding energies in eV: \n')
Exlog.write(str(Eb[:5]))
Exlog.close()

startTime = time.time()
sx=np.array([[0,1],[1,0]])
sy=np.array([[0,1.j],[-1.j,0]])

factorsigma=deg*kmesh.vcell*2*Ry*(a0/a)**2*(1/m)*1.j
omega=np.linspace(1.e-5,2.,1000)
eta=1.5e-2
csxv=np.zeros((circle.Np),dtype=np.complex)
xsx0=np.zeros((circle.Np),dtype=np.complex)
csyv=np.zeros((circle.Np),dtype=np.complex)
xsy0=np.zeros((circle.Np),dtype=np.complex)
dE=np.zeros((circle.Np),dtype=np.float64)

sigma=np.zeros((omega.shape[0]),dtype=np.complex128)
sigma0=np.zeros((omega.shape[0]),dtype=np.complex128)

csxv = moire.mdirac.cov_matrix_elements(sx,U,1,0)
xsx0 = moire.mdirac.exciton_elements(Ux,csxv)
csyv = moire.mdirac.cov_matrix_elements(sy,U,1,0)
xsy0 = moire.mdirac.exciton_elements(Ux,csyv)
dE = E[:,1]-E[:,0]

sigma0x=moire.mdirac.get_sigma_xx(factorsigma,omega+1.j*eta,dE,csxv)
sigmaxx=moire.mdirac.get_sigma_xx(factorsigma,omega+1.j*eta,Ex,xsx0)
sigma0y=moire.mdirac.get_sigma_xy(factorsigma,omega+1.j*eta,dE,csxv,csyv)
sigmaxy=moire.mdirac.get_sigma_xy(factorsigma,omega+1.j*eta,Ex,xsx0,xsy0)
sigma_rcp0_B = sigma0x+1.j*sigma0y
sigma_rcp_B = sigmaxx+1.j*sigmaxy
sigma_lcp0 = sigma0x-1.j*sigma0y
sigma_lcp = sigmaxx-1.j*sigmaxy

eps_B = 4*np.pi*sigma_rcp_B/omega*beta*1.j

executionTime = (time.time() - startTime)
timelog.write("Matrix element calculation: " + str(executionTime))
timelog.close()

###
### optical constants extractions and writing to file
###

Eg = 1.585+shiftA 

scaleomega=Eg/500

def SigmaPlotter(left_cut, right_cut,scaleomega,omega,sigma_A, sigma_B): 
    omegas = np.arange(omega.shape[0])[left_cut:right_cut]
    fig,ax=plt.subplots(dpi=120)
    # *4 to calculate in units of e2/4hbar
    # Resigma without excitons
    #ax.plot(scaleomega*omegas,
    #        (sigma_A.real[left_cut:right_cut]+sigma_B.real[left_cut:right_cut])*4,
    #        c='g',label='ReSigma wo x, rcp')
    ax.plot(scaleomega*omegas,
            (sigma_A.real[left_cut:right_cut]+sigma_B.real[left_cut:right_cut])*4,
            c='y',label=r'$\sigma_+$ pol')
    ax.set_xlabel(r'$\omega$, eV',fontsize=15)
    ax.grid()
    ax.set_ylabel(r'Re $\sigma$/$\sigma_0$',fontsize=15)
    lim = max(np.max(sigma_A.real[left_cut:right_cut]),
             np.max(sigma_B.real[left_cut:right_cut]))
    ax.vlines(x=[Eg], ymin=0,ymax=lim,colors=['tab:orange'],
              ls='--', lw=5, alpha=0.5,label='bandgap')
    ax.legend(loc='upper right')
    fig.tight_layout()
    realsigmapath = path + '/realsigma'+str(left_cut)+'_'+str(right_cut)+'.png'
    plt.savefig(realsigmapath,dpi=180)
    plt.close()
    """
    fig,ax=plt.subplots(dpi=120)
    ax.plot(scaleomega*omegas,sigma.imag[left_cut:right_cut]*4,c='r',label='ImSigma')
    ax.plot(scaleomega*omegas,sigma0.imag[left_cut:right_cut]*4,c='b',label='ImSigma wo x')
    ax.set_xlabel(r'$\omega$, eV',fontsize=15)
    ax.set_ylabel(r'Im $\sigma$/$\sigma_0$',fontsize=15)
    ax.vlines(x=[Eg], ymin=np.min(sigma0.imag)*4,ymax=np.max(sigma0.imag)*4,
              colors=['tab:orange'], ls='--', lw=5, alpha=0.5,label='opt bg')
    imsigmapath = path + '/imsigma'+str(left_cut)+'_'+str(right_cut)+'.png'
    ax.legend()
    plt.savefig(imsigmapath,dpi=120)
    sigmatofile = np.column_stack([Eg*omega[left_cut:right_cut], sigma.real[left_cut:right_cut]])
    np.savetxt(path+'/Resigma.txt',sigmatofile)
    plt.close()
    """
    """
    plot transmittance T = 1 - 4pi alpha Resigmaxx ???
    """
    """
    fig,ax=plt.subplots(dpi=120)
    ax.plot(scaleomega*omegas,
            1-4*np.pi*1/137*sigma_lcp0.real[left_cut:right_cut]*4,c='b',label='transmission wo x, lcp')
    ax.plot(scaleomega*omegas,
            1-4*np.pi*1/137*sigma_lcp.real[left_cut:right_cut]*4,c='r',label='transmission, lcp')
    ax.plot(scaleomega*omegas,
            1-4*np.pi*1/137*sigma_rcp0.real[left_cut:right_cut]*4,c='g',label='transmission wo x, rcp')
    ax.plot(scaleomega*omegas,
            1-4*np.pi*1/137*sigma_rcp.real[left_cut:right_cut]*4,c='y',label='transmission, rcp')
    ax.set_xlabel(r'$\omega$, eV',fontsize=15)
    ax.set_ylabel('Transmittance',fontsize=15)
    ax.vlines(x=[Eg], ymin=0,ymax=np.max(sigma0.real)*4,colors=['tab:orange'], ls='--', lw=5, alpha=0.5,label='opt bg')
    ax.set_ylim([0.4,1])
    ax.legend(loc='upper right')
    realsigmapath = path + '/transmission'+str(left_cut)+'_'+str(right_cut)+'.png'
    plt.savefig(realsigmapath,dpi=120)
    plt.close()
    """


def OptConstsPlotter(left_cut, right_cut,scaleomega,omega,eps_A,eps_B): 
    omegas = np.arange(omega.shape[0])[left_cut:right_cut]
    eps = 1+eps_A+eps_B
    refr = np.sqrt(eps)[left_cut:right_cut]

    fig,ax=plt.subplots(dpi=180)
    ax.plot(scaleomega*omegas,refr.real,c='#6C09C9',label='n')
    ax.plot(scaleomega*omegas,refr.imag,c='g',label='k')
    ax.set_xlim([scaleomega*omegas[0],scaleomega*omegas[-1]])
    ax.set_ylim([0,np.max(refr.real)+1])
    ax.set_xlabel(r'$\omega$, eV',fontsize=15)
    ax.grid()
    ax.legend(loc='upper right')
    fig.tight_layout()
    optconstspath = path + '/optconsts'+str(left_cut)+'_'+str(right_cut)+'.png'
    plt.savefig(optconstspath)
    conststofile = np.column_stack([Eg*omega[left_cut:right_cut], refr.real, refr.imag])
    np.savetxt(path+'/optconsts.txt',conststofile)
    plt.close()

    
SigmaPlotter(100,1000,scaleomega,omega,sigma_rcp_A, sigma_rcp_B)
SigmaPlotter(370,570,scaleomega,omega,sigma_rcp_A, sigma_rcp_B)
SigmaPlotter(300,650,scaleomega,omega,sigma_rcp_A, sigma_rcp_B)

OptConstsPlotter(100,1000,scaleomega,omega,eps_A,eps_B)
OptConstsPlotter(370,570,scaleomega,omega,eps_A,eps_B)
OptConstsPlotter(300,650,scaleomega,omega,eps_A,eps_B)

